In [13]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv('/content/sample_data/profanity.csv')
df = df[['sentence', 'label']]
df = df.dropna()

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
from sklearn.model_selection import train_test_split
import numpy as np
x = df['sentence'].to_list()
y = df['label'].to_list()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2, random_state=42)


In [17]:
import tensorflow_hub as hub
from tensorflow.keras.layers import TextVectorization
max_vocab = 10000
max_length = 20

text_vectorizer = TextVectorization(max_tokens=max_vocab,
                                    output_mode="int",
                                    output_sequence_length=max_length)
text_vectorizer.adapt(x_train)

In [18]:
class CustomTextVectorization(tf.keras.layers.Layer):
    def __init__(self, max_tokens, output_sequence_length, **kwargs):
        super(CustomTextVectorization, self).__init__(**kwargs)
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.text_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens,
            output_mode="int",
            output_sequence_length=output_sequence_length,
        )

    def call(self, inputs):
        return self.text_vectorizer(inputs)

custom_text_vectorizer = CustomTextVectorization(max_tokens=max_vocab, output_sequence_length=max_length)

In [19]:
from tensorflow.keras import layers
Embedder = layers.Embedding(input_dim=max_vocab,
                                     output_dim=20,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding")


In [20]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = Embedder(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="Model")

In [21]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

Epoch 1/100
5/5 [==============================] - 7s 254ms/step - loss: 0.6923 - accuracy: 0.5775 - val_loss: 0.6916 - val_accuracy: 0.5278
Epoch 2/100
5/5 [==============================] - 0s 37ms/step - loss: 0.6872 - accuracy: 0.6056 - val_loss: 0.6906 - val_accuracy: 0.4167
Epoch 3/100
5/5 [==============================] - 0s 41ms/step - loss: 0.6781 - accuracy: 0.6690 - val_loss: 0.6804 - val_accuracy: 0.6667
Epoch 4/100
5/5 [==============================] - 0s 36ms/step - loss: 0.6617 - accuracy: 0.8380 - val_loss: 0.6629 - val_accuracy: 0.7500
Epoch 5/100
5/5 [==============================] - 0s 39ms/step - loss: 0.6222 - accuracy: 0.8662 - val_loss: 0.6233 - val_accuracy: 0.7500
Epoch 6/100
5/5 [==============================] - 0s 36ms/step - loss: 0.5240 - accuracy: 0.9085 - val_loss: 0.4655 - val_accuracy: 0.8889
Epoch 7/100
5/5 [==============================] - 0s 41ms/step - loss: 0.2940 - accuracy: 0.9437 - val_loss: 0.2539 - val_accuracy: 0.9444
Epoch 8/100
5/5 [==

In [22]:
def final_pred(quote):
  val = model.predict([quote])
  # Round
  rounded = np.round(val)

  # Convert to integer
  a_int = rounded.astype(int)
  return a_int[0][0]

print(final_pred(['I love your mom']))

1/1 [==============================] - 1s 824ms/step
0


In [23]:
model.save('hate_speech.tf')